In [115]:
import pandas as pd

In [116]:
y_train = pd.read_csv('/workspaces/EDA_4/data/processed/heart_prevalence_y_train.csv')
y_test = pd.read_csv('/workspaces/EDA_4/data/processed/heart_prevalence_y_test.csv')
X_train_norm = pd.read_csv('/workspaces/EDA_4/data/interim/heart_prevalence_X_train_std.csv')
X_test_norm = pd.read_csv('/workspaces/EDA_4/data/interim/heart_prevalence_X_test_std.csv')

In [117]:
from sklearn.feature_selection import f_regression, SelectKBest

In [118]:
selection_model = SelectKBest(f_regression, k = 8)
selection_model.fit(X_train_norm, y_train)
ix = selection_model.get_support()
X_train_sel = pd.DataFrame(selection_model.transform(X_train_norm), columns = X_train_norm.columns.values[ix])
X_test_sel = pd.DataFrame(selection_model.transform(X_test_norm), columns = X_test_norm.columns.values[ix])

/home/vscode/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [119]:
X_train_sel["Heart disease_prevalence"] = y_train
X_test_sel["Heart disease_prevalence"] = y_test 

In [120]:
X_train_sel.to_csv("/workspaces/EDA_4/data/interim/heart_prevalence_X_train_sel.csv", index=False)
X_test_sel.to_csv("/workspaces/EDA_4/data/interim/heart_prevalence_X_test_sel.csv", index=False)

In [121]:
train_data = X_train_sel
test_data = X_test_sel
train_data.head()

,0,1,2,3,4,5,6,7,Heart disease_prevalence
0,1.592553,0.820420,-0.393446,-1.254632,-1.034310,-0.690028,-1.329247,-1.063396,7.321739
1,0.802857,0.113865,0.558434,-0.126974,-0.141938,-0.158183,-0.433800,-0.535918,8.031343
2,-0.644918,-0.311410,0.579479,0.363973,0.936350,-0.307709,0.011809,-0.196489,8.315238
3,0.210586,1.409531,0.635620,-1.011609,-1.099331,-0.350395,-1.043889,-1.534241,7.230000
4,0.342202,-1.292706,0.882003,0.003443,-0.337835,-0.487564,0.924241,1.998352,8.663636


In [122]:
X_train = train_data.drop(["Heart disease_prevalence"], axis = 1)
y_train = train_data["Heart disease_prevalence"]
X_train.head()

,0,1,2,3,4,5,6,7
0,1.592553,0.820420,-0.393446,-1.254632,-1.034310,-0.690028,-1.329247,-1.063396
1,0.802857,0.113865,0.558434,-0.126974,-0.141938,-0.158183,-0.433800,-0.535918
2,-0.644918,-0.311410,0.579479,0.363973,0.936350,-0.307709,0.011809,-0.196489
3,0.210586,1.409531,0.635620,-1.011609,-1.099331,-0.350395,-1.043889,-1.534241
4,0.342202,-1.292706,0.882003,0.003443,-0.337835,-0.487564,0.924241,1.998352


In [123]:
X_test = test_data.drop(["Heart disease_prevalence"], axis = 1)
y_test = test_data["Heart disease_prevalence"]
X_test.head()

,0,1,2,3,4,5,6,7
0,-0.381686,2.333274,-0.914804,-0.312357,-0.103458,-0.745705,-0.220811,-1.029929
1,-1.171382,3.201860,-3.699774,-2.592387,-2.434579,-2.193322,-2.465568,-2.489778
2,0.737049,-0.023037,1.816672,-0.233735,-0.412420,0.063166,-0.085710,0.403377
3,-1.368806,0.546627,0.280258,-1.980160,-2.283529,-1.204175,-1.573678,-0.723029
4,-0.447494,-0.386932,1.693048,-0.434702,-0.682066,0.068579,-0.628342,-0.081028


In [124]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [125]:
print(f"Intercept (a): {model.intercept_}")
print(f"Coefficients (b): {model.coef_}")

Intercept (a): 8.371498647349844
Coefficients (b): [-0.00516207 -0.1439993   0.16823499  0.7090782  -0.25306644  0.45160351
 -0.27693746  0.32185306]


In [126]:
y_pred = model.predict(X_test)
y_pred

array([7.08117536, 4.96290723, 8.79736245, 7.18008729, 8.75762955,
       9.08989626, 9.26294895, 6.18379142, 8.04285671, 6.45473266,
       8.23749087])

In [127]:
from sklearn.metrics import mean_squared_error, r2_score
import math

print(f"Error cuadrático medio: {math.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"Coeficiente de determinación: {r2_score(y_test, y_pred)}")

Error cuadrático medio: 0.3660827391884049
Coeficiente de determinación: 0.9362860918269158
